In [1]:
#We are working together on this via vscode live share and are talking over zoom. 
#In terms of sharing the file we have a github repository set up.
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
from datetime import datetime as dt
import numpy as np

In [2]:
#2
url = "https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares"
webpage = requests.get(url)
print(webpage) #response 403, this means that the webserver refused to authorize the request
#to fix do this
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
webpage = requests.get(url, headers=headers)
print(webpage) #now response 200
soup_content = BeautifulSoup(webpage.content,'html.parser')
pretty = soup_content.prettify()
#print(pretty)
table_html = soup_content.find("table",{"class":"table table-striped"})#['data-value'] #from stackoverflow
df = pd.read_html(table_html.prettify())[0]
df.rename(columns={'Unnamed: 0':"rank",'Unnamed: 1':"x_class",'Unnamed: 2':"date",'Start':"Start_Time",'Maximum':"Max_Time",'End':"End_Time",'Unnamed: 7':"Movie"},inplace=True)
df.head()


<Response [403]>
<Response [200]>


,rank,x_class,date,Region,Start_Time,Max_Time,End_Time,Movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,Movie View archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,Movie View archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,Movie View archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,Movie View archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,Movie View archive


In [3]:
dataFrame = df.drop("Movie",1)
dataFrame.head()

for row in dataFrame.iterrows():
    date = pd.to_datetime(row[1].date)
    time_start = pd.to_datetime(row[1].Start_Time)
    str_time = str(date)[:11]+str(time_start)[11:]
    startTime = pd.to_datetime(str_time)
    dataFrame.at[row[0],'Start_Time'] = startTime
    
    time_max= pd.to_datetime(row[1].Max_Time)
    str_time = str(date)[:11]+str(time_max)[11:]
    maxTime = pd.to_datetime(str_time)
    dataFrame.at[row[0],'Max_Time'] = maxTime
    
    time_end = pd.to_datetime(row[1].End_Time)
    str_time = str(date)[:11]+str(time_end)[11:]
    endTime = pd.to_datetime(str_time)
    dataFrame.at[row[0],'End_Time'] = endTime
    #date_time_obj = dt.strftime(str_time, '%y-%m-%d %H:%M:%S')
    #dt.combine(date,time_start)
dataFrame = dataFrame.replace('-',np.nan)
dataFrame_update = dataFrame.drop("date",1)
dataFrame_update.rename(columns={'Start_Time':'start_datetime','Max_Time':'max_datetime','End_Time':'end_datetime'},inplace=True)
region_column = dataFrame_update.Region
dataFrame_update=dataFrame_update.drop(['Region'],axis=1)
dataFrame_update['region'] = region_column
print(dataFrame_update.dtypes)
dataFrame_update


rank               int64
x_class           object
start_datetime    object
max_datetime      object
end_datetime      object
region             int64
dtype: object


/tmp/ipykernel_530/1481140437.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataFrame = df.drop("Movie",1)
/tmp/ipykernel_530/1481140437.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataFrame_update = dataFrame.drop("date",1)


,rank,x_class,start_datetime,max_datetime,end_datetime,region
0,1,X28+,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00,486
1,2,X20+,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00,9393
2,3,X17.2+,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00,486
3,4,X17+,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00,808
4,5,X14.4,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00,9415
5,6,X10,2003-10-29 20:37:00,2003-10-29 20:49:00,2003-10-29 21:01:00,486
6,7,X9.4,1997-11-06 11:49:00,1997-11-06 11:55:00,1997-11-06 12:01:00,8100
7,8,X9.3,2017-09-06 11:53:00,2017-09-06 12:02:00,2017-09-06 12:10:00,2673
8,9,X9,2006-12-05 10:18:00,2006-12-05 10:35:00,2006-12-05 10:45:00,930
9,10,X8.3,2003-11-02 17:03:00,2003-11-02 17:25:00,2003-11-02 17:39:00,486


In [11]:
#Step 3
url = "http://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
webpage = requests.get(url, headers=headers)
soup_content = BeautifulSoup(webpage.content,'html.parser')
pretty = soup_content.prettify()
row_split = soup_content.find("pre").prettify().split('\n')
colomns = []
data = []
for row in row_split[14:15]:
    data_split = row.split(' ')
    print(data_split)
    #####
    cme_date = data_split[19].split('>')[1][:-3] #cme
    cme_time = data_split[20]
    cpa = data_split[23]
    end_date = data_split[2]
    end_frequency = int(data_split[9].split('>')[1][:-3])
    end_time = data_split[3]
    flare_location = data_split[12]
    flare_region = data_split[14]
    importance = data_split[15]
    speed =  int(data_split[29].split('>')[1][:-3])
    start_date = data_split[0]
    start_frequency = int(data_split[6].split('>')[1][:-3])
    start_time = data_split[1]
    width = data_split[26]
    data.append([cme_date,cme_time,cpa,end_date,end_frequency,end_time,
                flare_location,flare_region,importance,speed,start_date,
                start_frequency,start_time,width])
columns = ["cme_date","cme_time","cpa","end_date","end_frequency","end_time",
                "flare_location","flare_region","importance","speed","start_date",
                "start_frequency","start_time","width"]
nasa_df = pd.DataFrame(data,columns = columns)
nasa_df
    
    
    
#pretty

['1997/05/12', '05:15', '05/14', '16:00', '<a', 'href="https://cdaw.gsfc.nasa.gov/CME_list/daily_movies/1997/05/12/c2rdif_waves.html">12000</a>', '', '', '', '<a', 'href="https://cdaw.gsfc.nasa.gov/CME_list/daily_movies/1997/05/12/c3rdif_waves.html">80</a>', '', '', 'N21W08', '', '8038', 'C1.3', '', '', '<a', 'href="https://cdaw.gsfc.nasa.gov/CME_list/UNIVERSAL/1997_05/jsmovies/1997_05/19970512.053005.p264g/c2_rdif.html">05/12</a>', '05:30', 'Halo', '', '360', '', '<a', 'href="https://cdaw.gsfc.nasa.gov/CME_list/UNIVERSAL/1997_05/htpng/19970512.053005.p264g.htp.html">464</a>', '', '', '<a', 'href="https://cdaw.gsfc.nasa.gov/CME_list/daily_plots/sephtx/1997_05/sephtx_19970512.png">PHTX</a>']


IndexError: list index out of range